In [ ]:
!pip install codecarbon


In [ ]:
# ===============================================================
# BASELINE MODEL BLOCK (Clean version)
# ===============================================================

from codecarbon import EmissionsTracker
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch, time, csv, os
from datetime import datetime

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on:", device)

# === CONFIG ===
model_name = "deepseek-ai/deepseek-coder-1.3b-base"
output_dir = "/content/results"
os.makedirs(output_dir, exist_ok=True)
csv_path = os.path.join(output_dir, "baseline_results.csv")

# === HELPER FUNCTIONS ===
def evaluate_accuracy(text):
    checks = ['"""', "Args:", "Returns:"]
    return "P" if all(x in text for x in checks) else "F"

def get_model_size_mb(model):
    torch.save(model.state_dict(), "temp.pt")
    size = os.path.getsize("temp.pt") / (1024 * 1024)
    os.remove("temp.pt")
    return round(size, 2)

def run_inference(model, tokenizer, prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    start = time.perf_counter()
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.3,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    latency = round(time.perf_counter() - start, 3)
    text = tokenizer.decode(output[0], skip_special_tokens=True)
    return text, latency

# === BASELINE TASK PROMPT ===
code_sample = """from typing import List
def below_zero(operations: List[int]) -> bool:
    balance = 0
    for op in operations:
        balance += op
        if balance < 0:
            return True
    return False
"""

prompt_text = (
    "You are an expert Python developer. "
    "Given the function below, generate a complete and well-formatted Google-style docstring. "
    "Include purpose, parameters, returns, and one example. "
    "Provide only the full function code with the docstring.\n\n"
    f"{code_sample}"
)

# === LOAD MODEL ===
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# === RUN BASELINE EXPERIMENT ===
EPOCHS = 5

if not os.path.exists(csv_path):
    with open(csv_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Model", "Compression", "Epoch", "CO2_kg", "Latency_s", "Model_Size_MB", "Accuracy", "Timestamp"])

for epoch in range(1, EPOCHS + 1):
    print(f"\n🚀 Epoch {epoch}...")
    tracker = EmissionsTracker(project_name="Baseline", output_dir=output_dir)
    tracker.start()

    generated, latency = run_inference(model, tokenizer, prompt_text)
    co2 = tracker.stop()

    size = get_model_size_mb(model)
    acc = evaluate_accuracy(generated)

    with open(csv_path, "a", newline="") as f:
        csv.writer(f).writerow(
            ["Baseline", "None", epoch, co2, latency, size, acc, datetime.now()]
        )

print(f"\n✅ Baseline experiment complete. Results saved to: {csv_path}")


Running on: cuda


[codecarbon WARNING @ 17:19:39] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 17:19:39] [setup] RAM Tracking...
[codecarbon INFO @ 17:19:39] [setup] CPU Tracking...



🚀 Epoch 1...


[codecarbon WARNING @ 17:19:40] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 17:19:40] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 17:19:40] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon WARNING @ 17:19:40] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 17:19:40] [setup] GPU Tracking...
[codecarbon INFO @ 17:19:40] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 17:19:40] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: pynvml
            
[codecarbon INFO @ 17:19:40] >>> Tracker's metadata:
[codecarbon INFO @ 17:19:40]   Platform sys


🚀 Epoch 2...


...


🚀 Epoch 4...


[codecarbon WARNING @ 17:27:17] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 17:27:17] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 17:27:17] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon WARNING @ 17:27:17] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 17:27:17] [setup] GPU Tracking...
[codecarbon INFO @ 17:27:17] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 17:27:17] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: pynvml
            
[codecarbon INFO @ 17:27:17] >>> Tracker's metadata:
[codecarbon INFO @ 17:27:17]   Platform sys


🚀 Epoch 5...


[codecarbon WARNING @ 17:29:34] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 17:29:34] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 17:29:34] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon WARNING @ 17:29:34] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 17:29:34] [setup] GPU Tracking...
[codecarbon INFO @ 17:29:34] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 17:29:34] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: pynvml
            
[codecarbon INFO @ 17:29:34] >>> Tracker's metadata:
[codecarbon INFO @ 17:29:34]   Platform sys


✅ Baseline experiment complete. Results saved to: /content/results/baseline_results.csv
